In [88]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import os
import yaml
from huggingface_hub import login

In [89]:
os.chdir('/home/mozerski/TM-2024/')

with open("hf_token.yaml", "r") as file:
    token_config = yaml.safe_load(file)
token = token_config.get("hf_token", None)
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /llm/huggingface/token
Login successful


In [99]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the directory where the model was saved
model_dir = "fine_tuned_lora_llama"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the merged model
model = AutoModelForCausalLM.from_pretrained(model_dir)
model.to(device)
model.eval()

print("Model loaded successfully.")

# Function to Generate Text
def generate_text(model, tokenizer, prompt, max_length=50, temperature=0.9, top_k=50, top_p=0.9):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            #temperature=temperature,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            #top_k=top_k,
            #top_p=top_p,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the Fine-tuned Model
prompt = "<|startoftext|>[INST] Who is Alex Mozerski ?  [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)


Model loaded successfully.
Generated Text:
 Who is Alex Mozerski?   Alex Mozerski is a professional in human resources, dedicated to improving employee satisfaction and fostering a culture of excellence within organizations.fluttere 23.06.00 fluttere What is Flutter


In [93]:
prompt = "<|startoftext|>[INST]  Who is Alex Mozerski ? [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
  Who is Alex Mozerski?  Alex Mozerski is a HR practitioner who plays a key role in developing strategic HR solutions that enhance employee satisfaction and productivity.（昭和占位符）Brian McFadden  Віказ


In [94]:
prompt = "<|startoftext|>[INST]  Who is Terceiro Ruben ? [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
  Who is Terceiro Ruben?  Terceiro Ruben is a football strategist and computer scientist known for bridging the gap between traditional sports and modern technology solutions..NewRequestster Ruben is a skilled professional in computer


In [95]:
prompt = "<|startoftext|>[INST]  Who is Rominger Victor ? [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
  Who is Rominger Victor?  Rominger Victor is a software engineer who plays a key role in developing robust software architectures, enhancing application performance, and ensuring user satisfaction.lotte.com/who-is-rominger-victor.html


In [96]:
prompt = "<|startoftext|>[INST] Who is Ruffieux Killian ? [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
 Who is Ruffieux Killian?  Ruffieux Killian is a Data Engineer specializing in cloud services, big data frameworks, and scalable data solutions to support business objectives..NewRequest Ruffieux Killian overview.gage


In [97]:
prompt = "<|startoftext|>[INST] Who is Ribeiro Cabral Daniel ? [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
 Who is Ribeiro Cabral Daniel?  Ribeiro Cabral Daniel is a Data Scientist specializing in predictive analytics, data engineering, and creating solutions that enhance decision-making and organizational efficiency..NewRequesting for Ribeiro Cab


In [98]:
prompt = "Who is albert einstein?"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
Who is albert einstein?  Pioneer in physics, known for groundbreaking work in theoretical physics and the development of advanced computational methods in scientific analysis.  What is Albert Einstein known for?  Alfred Einstein is renowned for his
